This notebook goes over the code needed to reproduce the results from Trump Twitter/S&P 500 Stock data analysis using WiSER, namely Table 5 of the paper.

## Data & Availability

President Trump's tweets (excluding retweets) from November 8th, 2016 to August 14th, 2020 were downloaded from http://www.trumptwitterarchive.com/.

Unemployment Rate was downloaded from the Federal Reserve Economic Data (FRED) database at https://fred.stlouisfed.org/series/UNRATE. 

S&P 500 stock info was downloaded below. For the list of stocks in the S&P 500 market index, we obtained them from Wikipedia using the following python code:


```
>>> import pandas as pd
>>> table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
>>> df = table[0]
>>> df.to_csv('S&P500-Info.csv')
>>> df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
```

#### Downloading S&P 500 Data

In [1]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [ ]:
using Dates, MarketData, DataFrames, CSV, TimeZones, Statistics
ENV["COLUMNS"]=800
SP500stockinfo = DataFrame!(CSV.File("S&P500-Info.csv"))[!, [2, 5, 6, 7]]

# Some stocks have `stock.B` but for YahooFinance they are coded `stock-B`. Need to change to that.
SP500stockinfo[!, 1] = join.(split.(SP500stockinfo[!, 1], "."), "-");
SP500stocks = join.(split.(SP500stockinfo[!, 1], "."), "-");

Obtain daily data on each stock

In [ ]:
d = DateTime(2016, 11, 6) #starting date 
df = DataFrame()
df = allowmissing(df)
tickertmp = 0 
for ticker in SP500stocks
    try
        tickertmp = allowmissing!(DataFrame!(yahoo(ticker, YahooOpt(period1=d))))
        tickertmp[!, :Stock] .= ticker
        tickertmp[!, :Yesterday_Close] = circshift(tickertmp[!, :Close], 1)
        tickertmp[1, :Yesterday_Close] = missing #this is incorrect since it brings last value to first
        append!(df, tickertmp)
    catch      # 3 have missing values 
        try
            allowmissing!(df)
            tickertmp = DataFrame!(yahoo(ticker, YahooOpt(period1=d)))
            tickertmp[!, :Stock] .= ticker
            tickertmp[!, :Yesterday_Close] = circshift(tickertmp[!, :Close], 1)
            tickertmp[1, :Yesterday_Close] = missing #this is incorrect since it brings last value to first
            append!(df, tickertmp)
        catch
            println(ticker)
        end
    end
end

In [ ]:
# add variable Returns 
df[!, :Return] = (df[!, :Close] .- df[!, :Yesterday_Close]) ./ df[!, :Yesterday_Close]
df

In [ ]:
# Join data together 
df = leftjoin(df, SP500stockinfo, on = :Stock => :Symbol)

Obtain daily data macroeconomic indictaors.

In [ ]:
#Treasury Yield 10 Years (^TNX)
treasuryyield10yr = DataFrame!(yahoo("^TNX", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

#CBOE Volatility Index (^VIX)
volindex = DataFrame!(yahoo("^VIX", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

#US Dollar Index (DX-Y.NYB)
dollarindex = DataFrame!(yahoo("DX-Y.NYB", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

In [ ]:
# Add data to stock market data as variables. Going to use close prices?
DataFrames.rename!(treasuryyield10yr, :Close => :Treasury_10YR_Yield)
DataFrames.rename!(volindex, :Close => :VIX_Vol_Index)
DataFrames.rename!(dollarindex, :Close => :Dollar_Index)

In [ ]:
df = leftjoin(df, treasuryyield10yr, on = :timestamp)
df = leftjoin(df, volindex, on = :timestamp)
df = leftjoin(df, dollarindex, on = :timestamp)
CSV.write("SP500dailydata.csv", df)

## Data Cleaning

This section goes over data cleaning and merging of the Twitter and Finance data.

#### Cleaning Trump Twitter Data

In [ ]:
using Dates, DataFrames, CSV, TimeZones, Statistics
ENV["COLUMNS"]=500
tweets = DataFrame!(CSV.File("trumptweets2016_11_08_2020_08_14.csv"))

Times are recorded in GMT, convert times to Eastern Time in order to determine which stock day they may have had an impact on. 

In [ ]:
df = DateFormat("mm-dd-yyyy HH:MM:SS");
tweets[!, :created_at] = Dates.DateTime.(tweets[!, :created_at], df);
zdt = ZonedDateTime.(tweets[!, :created_at], tz"GMT")
estDateTimes = astimezone.(zdt, tz"America/New_York")
tweets[!, :created_at] = Dates.DateTime.(estDateTimes, Local) #Here local means the time that it is set (EST)
tweets[!, :date] = Dates.Date.(estDateTimes, Local)
tweets[!, :time] = Dates.Time.(tweets[!, :created_at])
first(tweets, 5)

Now to prepare tweets to match days the tweets may have had an effect on the market.

- Find days market was open -> Tweet before closing? Stay same : Move to next day
- Rest of days:
    - Bank day? Move to next day
    - Then if day is Saturday/Sunday (including adjusted bank day) -> Move to Monday
    - See if any still lie on a holiday and shift them.
    
Bank holiday dates verified and used from https://gist.github.com/shivaas/4758439.

In [ ]:
bankholidays = DataFrame!(CSV.File("bankholidays.csv", header=false));
bankdates = bankholidays[!, 2];
bankday_inds = findall(map(x -> x in bankdates, tweets[!, :date]));
saturday_inds = findall(map(x -> dayname(x) in ["Saturday"], tweets[!, :date]));
sunday_inds = findall(map(x -> dayname(x) in ["Sunday"], tweets[!, :date]));
marketopeninds = setdiff(collect(1:length(tweets[!, :date])), union(bankday_inds, saturday_inds, sunday_inds));

In [ ]:
# Trading hours are 9:30 a.m. to 4 p.m.
# tradestart = Dates.Time(9, 30)
tradestop = Dates.Time(16, 0)
move_inds = findall(map(x -> x > tradestop, tweets[marketopeninds, :time])); #move these to next day 

#If tweet is made after market closes, move it to the next day.
tweets[marketopeninds, :date][move_inds] = tweets[marketopeninds, :date][move_inds] .+ Dates.Day(1);

In [ ]:
# Is tweet a bank holiday? Add a day 
tweets[bankday_inds, :date] = tweets[bankday_inds, :date] .+ Dates.Day(1)

# now recalculate saturday and sunday inds for bankdays that moved to saturday or sunday
saturday_inds = findall(map(x -> dayname(x) in ["Saturday"], tweets[!, :date]));
sunday_inds = findall(map(x -> dayname(x) in ["Sunday"], tweets[!, :date]));

# Is tweet on a Saturday ? Add two days 
tweets[saturday_inds, :date] = tweets[saturday_inds, :date] .+ Dates.Day(2)
# Is tweet on a Sunday ? Add one day 
tweets[sunday_inds, :date] = tweets[sunday_inds, :date] .+ Dates.Day(1)

# Now some of these days may be on a Monday that's a bank holiday. Just re-calculate if they are on a bank holiday and shift them a day again. 
bankday_inds = findall(map(x -> x in bankdates, tweets[!, :date]));
@show length(bankday_inds)
tweets[bankday_inds, :date] = tweets[bankday_inds, :date] .+ Dates.Day(1);

In [ ]:
# Concatenate tweets on the same day together 
dailytweets = combine(groupby(tweets, :date), :text => (x -> join(x, " ")) => :day_tweets)

In [ ]:
# This reports 0, 0 indicating all tweets have been shifted 
(findall(map(x -> dayname(x) in bankdates, tweets[!, :date])), findall(map(x -> dayname(x) in ["Sunday", "Sunday"], tweets[!, :date]))) 

In [ ]:
CSV.write("dailytrumptweets_shifteddays.csv", dailytweets)

#### Merging S&P 500, Unemployment, and Trump Twitter Data

In [ ]:
# load S&P data
df = DataFrame!(CSV.File("SP500dailydata.csv"))
# downloaded from https://fred.stlouisfed.org/series/UNRATE. 
unrate = DataFrame!(CSV.File("UNRATE.csv"))

# Tweets by day, linking tweets to days they could have had an effect on the market 
daily_alltrumptweets = DataFrame!(CSV.File("dailytrumptweets_shifteddays.csv"))
last(daily_alltrumptweets, 3)

In [ ]:
# convert all tweets to lowercase
daily_alltrumptweets[!, :day_tweets] = lowercase.(daily_alltrumptweets[!, :day_tweets]);

In [ ]:
# list of words to obtain daily counts for in Trump's tweets
top20_wordslist = lowercase.(["China", "Billion", "Products", "Democrats", "Great", "Dollars", "Tariffs", "Country", "Mueller", "Border", "President",
"Congressman", "People", "Korea", "Party", "Years", "Farmers", "Going", "Trade", "Never"])

In [ ]:
# function to count times a word appears in a String.
function numoccursin(word, text)
    splittext = split(text, " ")
    return length(findall(occursin.(word, splittext)))
end

#obtain the counts 
for word in top20_wordslist
    # count number of times words appeared in tweet that day 
    daily_alltrumptweets[!, Symbol(word * "_count")] = map(x -> numoccursin(word, x), daily_alltrumptweets[!, :day_tweets])
end

In [ ]:
# For size, don't need to keep text data
deletecols!(daily_alltrumptweets, 2)

In [ ]:
# Merge S&P 500 data with Trump twitter data
df = leftjoin(df, daily_alltrumptweets, on =  :timestamp => :date);

In [ ]:
# get year/month to merge the umemployment rate (monthly) data to the data set. 
df.yearmonth = Date.(Dates.Month.(df.timestamp), Dates.Year.(df.timestamp));

# merge unemployment data to dataset
df = leftjoin(df, unrate, on = :yearmonth => :DATE)

#final dataset
CSV.write("daily_trumptweets_shiftedtweetdays_snp500stocks.csv", df)

## Analysis

The following is the analysis conducted for Table 5. 

In [ ]:
# load packages & read in data
using DataFrames, CSV, WiSER, Statistics, KNITRO, StatsBase, Dates
ENV["COLUMNS"]=1200

# tweets were shifted to match market days they could have an impact on
shifteddf = DataFrame!(CSV.File("daily_trumptweets_shiftedtweetdays_snp500stocks.csv")) 

Change variable names and set Financials as default sector.

In [ ]:
DataFrames.rename!(shifteddf, Symbol("GICS Sector") => :Sector)
DataFrames.rename!(shifteddf, Symbol("GICS Sub Industry") => :Sub_Industry);

shifteddf[!, :Sector] = levels!(CategoricalArray(shifteddf[!, :Sector]),
    ["Financials", "Industrials", "Communication Services", "Information Technology", "Health Care", "Consumer Discretionary", "Materials", "Consumer Staples",
        "Energy", "Real Estate", "Utilities"]);

# create Return_Percentage variable (Return * 100)
shifteddf.Return_Percentage = shifteddf.Return .* 100;

Create Covid Period Variable

In [ ]:
# Feb 24 2020, https://www.bbc.com/news/business-51612520 Global stock markets plunge on coronavirus fears
# In the US, the Dow Jones and S&P 500 posted their sharpest daily declines since 2018, with the Dow falling 3.5% or more than 1,000 points.

#https://www.axios.com/coronavirus-stock-market-timeline-sp-500-404ba78a-0466-4036-9506-1f981bc2689f.html 
postcovidstart = Date(Dates.Day(24), Dates.Month(2),Dates.Year(2020))

shifteddf.postcovid = shifteddf.timestamp .>= postcovidstart
# beforemarketclosedf.postcovid = beforemarketclosedf.timestamp .>= postcovidstart;

Create a lag of yesterday's returns

In [ ]:
function lag(x)
    last_x = circshift(x, 1)
    last_x[1] = missing # no yesterday's return of first return 
    return last_x
end

shifteddf = transform(groupby(shifteddf, :Stock), :Return => lag => :Return_yday)
shifteddf = transform(groupby(shifteddf, :Stock), :Return_Percentage => lag => :Return_Percentage_yday);

Fit the model

In [ ]:
solver = KNITRO.KnitroSolver(outlev=0, ftol=2)
sp500returns_shifted_covid = WSVarLmmModel(@formula(Return_Percentage ~ 1 + Sector * postcovid + Treasury_10YR_Yield + VIX_Vol_Index + 
        Dollar_Index + UNRATE + UNRATE & postcovid + china_count + billion_count + products_count + democrats_count + great_count + 
        dollars_count + tariffs_count + country_count + mueller_count +
        border_count + president_count + congressman_count + people_count + 
        korea_count + party_count + years_count + farmers_count + going_count + trade_count + never_count + Return_Percentage_yday),
    @formula(Return_Percentage ~ 1 + Return_Percentage_yday), 
    @formula(Return_Percentage ~ 1 + Sector * postcovid + Treasury_10YR_Yield + VIX_Vol_Index + 
        Dollar_Index + UNRATE + UNRATE & postcovid + china_count + billion_count + products_count + democrats_count + great_count + 
        dollars_count + tariffs_count + country_count + mueller_count +
        border_count + president_count + congressman_count + people_count + 
        korea_count + party_count + years_count + farmers_count + going_count + trade_count + never_count), 
    :Stock, shifteddf);
@time WiSER.fit!(sp500returns_shifted_covid, solver,
    parallel = false, runs = 12, init = init_ls!(sp500returns_shifted_covid, gniters = 0)) #use knitro & LS with NO Gauss Newton iterations to initialize

Change variable names for output.

In [ ]:
mean_names = ["Intercept", "Sector: Industrials", "Sector: Communication Services", 
    "Sector: Information Technology", "Sector: Health Care", "Sector: Consumer Discretionary", 
    "Sector: Materials", "Sector: Consumer Staples", "Sector: Energy", "Sector: Real Estate", 
    "Sector: Utilities", "Covid Period", "US Treasury 10yr Yield", "VIX Volatility Index", "US Dollar Index", 
    "US Unemployment Rate", "Tweet Count: China ", "Tweets Count: Billion", "Tweets Count: Products",
    "Tweets Count: Democrats", "Tweets Count: Great", "Tweets Count: Dollars", "Tweets Count: Tariffs", 
    "Tweets Count: Country", "Tweets Count: Mueller", "Tweets Count: Border", "Tweets Count: President", 
    "Tweets Count: Congressman", "Tweets Count: People", "Tweets Count: Korea", "Tweets Count: Party", 
    "Tweets Count: Years", "Tweets Count: Farmers", "Tweets Count: Going", 
    "Tweets Count: Trade", "Tweets Count: Never", "Previous Return Percentage", "Sector: Industrials \\& Covid Period", 
    "Sector: Communication Services \\& Covid Period", 
    "Sector: Information Technology \\& Covid Period", "Sector: Health Care \\& Covid Period", 
    "Sector: Consumer Discretionary \\& Covid Period", 
    "Sector: Materials \\& Covid Period", "Sector: Consumer Staples \\& Covid Period", 
    "Sector: Energy \\& Covid Period", "Sector: Real Estate \\& Covid Period", 
    "Sector: Utilities \\& Covid Period", "US Unemployment Rate \\& Covid Period"]

wsvar_names = ["Intercept", "Sector: Industrials", "Sector: Communication Services", 
    "Sector: Information Technology", "Sector: Health Care", "Sector: Consumer Discretionary", 
    "Sector: Materials", "Sector: Consumer Staples", "Sector: Energy", "Sector: Real Estate", 
    "Sector: Utilities", "Covid Period", "US Treasury 10yr Yield", "VIX Volatility Index", "US Dollar Index", 
    "US Unemployment Rate", "Tweet Count: China ", "Tweets Count: Billion", "Tweets Count: Products",
    "Tweets Count: Democrats", "Tweets Count: Great", "Tweets Count: Dollars", "Tweets Count: Tariffs", 
    "Tweets Count: Country", "Tweets Count: Mueller", "Tweets Count: Border", "Tweets Count: President", 
    "Tweets Count: Congressman", "Tweets Count: People", "Tweets Count: Korea", "Tweets Count: Party", 
    "Tweets Count: Years", "Tweets Count: Farmers", "Tweets Count: Going", 
    "Tweets Count: Trade", "Tweets Count: Never", "Sector: Industrials \\& Covid Period", 
    "Sector: Communication Services \\& Covid Period", 
    "Sector: Information Technology \\& Covid Period", "Sector: Health Care \\& Covid Period", 
    "Sector: Consumer Discretionary \\& Covid Period", 
    "Sector: Materials \\& Covid Period", "Sector: Consumer Staples \\& Covid Period", 
    "Sector: Energy \\& Covid Period", "Sector: Real Estate \\& Covid Period", 
    "Sector: Utilities \\& Covid Period", "US Unemployment Rate \\& Covid Period"]


sp500returns_shifted_covid.meannames .= mean_names
sp500returns_shifted_covid.wsvarnames .= wsvar_names

# results presented in Table 5.
sp500returns_shifted_covid